# Generate three dfs

- df_handlabled    --- handlabeled xlsx in data_cycle2 on s3 (uploaed from google drive directly)
- df_original --- merge( train and test in data_cycle1 on s3)
- df_temp --- 

In [1]:
# # Install a pip package in the current Jupyter kernel
# import sys
# !{sys.executable} -m pip install s3fs

import numpy as np
import pandas as pd
import cPickle
from collections import defaultdict
import re
import sys
import os
from sklearn.model_selection import train_test_split

URL0 = 's3://smart-newsdev-dmp/tmp/data/classification/data_handlabeling_cycle2/error_handlabeled_combined.xlsx'
URL1 = 's3://smart-newsdev-dmp/tmp/data/classification/data_handlabeling_cycle1/train.csv'
URL2 = 's3://smart-newsdev-dmp/tmp/data/classification/data_handlabeling_cycle1/test.csv'

In [2]:
def get_datasheet(url, sheetname):
    # extract specific sheet from google doc
    xls = pd.ExcelFile(url)
    return pd.read_excel(xls, sheetname)

def deduplicate(df, columnlist):
    
    df.drop_duplicates(columnlist,inplace=True)
    return df

sheetname = 'error_combined (1).csv'
df_handlabeled = get_datasheet(URL0, sheetname)
print(df_handlabeled.shape)

(2110, 7)


In [3]:
def clean_idx(data):
    data = data[data['idx'].notnull()]
    data['idx'] = data['idx'].map(int)
    
    return data
    
def generate_original_df(URL1, URL2):

    df1 = pd.read_csv(URL1, sep='|')
    df2 = pd.read_csv(URL2, sep='|')

    df_original = pd.concat([df1, df2])
    
    return df_original

df_original = generate_original_df(URL1, URL2)
print (df_original.shape)

(9855, 7)


In [4]:
def merge_original_handlabeled(df_original, df_handlabeled):
    
    # remove NA
    df_handlabeled = df_handlabeled[df_handlabeled.Re_category.notnull()]
    
    # deduplicate
    df_handlabeled = deduplicate(df_handlabeled, ['idx'])
    df_original = deduplicate(df_original, ['idx'])
    
    # df_original_kept(merge later)
    df_original_kept = df_original[~df_original['idx'].isin(df_handlabeled['idx'].values)]
    print (df_original_kept.shape)

    # merge
    df_merged = pd.merge(df_original[df_original['idx'].isin(df_handlabeled['idx'].values)], 
                         df_handlabeled.loc[:,['idx','Re_category']], how ='left', left_on=['idx'], right_on=['idx'])
    
    # remove baddata
    df_merged = df_merged[df_merged.Re_category != 'BADDATA']
    
    # drop column
    df_merged.drop(['category'], axis=1, inplace=True)
    
    # rename column
    df_merged.rename(columns={'Re_category': 'category'}, inplace=True)
    
    # merge horizontally
    df_merged = pd.concat([df_original_kept, df_merged])
    
    del df_original_kept
    
    return df_merged


df_temp = merge_original_handlabeled(df_original, df_handlabeled)
print(df_temp.shape)

(9670, 7)
(9830, 7)


/home/ec2-user/anaconda3/envs/tensorflow_p27/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


# Generate train, test data

In [5]:
df_temp = df_temp[df_temp['category'] != 'UNKNOWN']
print(df_temp.shape)

train, test = train_test_split(df_temp, test_size=0.2, random_state=42)
print (train.category.value_counts())

(9830, 7)
EN_US_SPORTS           1102
EN_US_ENTERTAINMENT     977
EN_US_SCIENCE           955
EN_US_POLITICS          939
EN_US_WORLD             936
EN_US_TECHNOLOGY        914
EN_US_NATIONAL          761
EN_US_LIFESTYLE         715
EN_US_BUSINESS          565
Name: category, dtype: int64


# Move files to s3

In [6]:
train.to_csv('train.csv', sep='|', index = False)
test.to_csv('test.csv', sep='|', index = False)

In [7]:
from io import StringIO
import boto3

def upload_to_s3(bucket, folder, file):
    s3 = boto3.resource('s3')
    data = open(file, "rb")
    key = 'tmp/data/classification/' + folder + file
    s3.Bucket(bucket).put_object(Key=key, Body=data)

bucket = 'smart-newsdev-dmp'
folder = 'data_handlabeling_cycle2/' ############### double-check

upload_to_s3(bucket, folder, 'train.csv')
upload_to_s3(bucket, folder, 'test.csv')

# Manually add tag (SageMaker = true) on the new files in s3